In [2]:
!pip install numpy
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163007 sha256=69e0e7ac9c4bbed6a65801013862f3c7ca4cb7831341c1f69c688a1def7ac135
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [36]:
import sklearn

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
%matplotlib inline

from pathlib import Path

import heapq
from collections import defaultdict

import pandas as pd
import matplotlib.pylab as plt
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split

In [4]:
#Class ID reference
#1 SQL
#2 Spatial
#3 PA1
#4 DMinR
#5 Python
#6 Forecast
#7 RProg
#8 Hadoop
#9 Regression

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
#Name ID Reference
#1 LN
#2 MH
#3 JH
#4 EN
#5 DU
#6 FL
#7 GL
#8 AH
#9 SA
#10 RW
#11 BA
#12 MG
#13 AF
#14 KG
#15 DS

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
ratings = pd.DataFrame([
    [1,1,4],[1,5,3],[1,6,2],[1,7,4],[1,9,2],
    [2,1,3],[2,2,4],[2,5,4],
    [3,1,2],[3,2,2],
    [4,1,4],[4,4,4],[4,7,4],[4,9,3],
    [5,1,4],[5,2,4],
    [6,2,4],
    [7,2,4],
    [8,2,3],
    [9,3,4],
    [10,3,2],[10,8,4],
    [11,3,4],
    [12,3,4],[12,6,4],
    [13,3,4],
    [14,3,3],
    [15,1,4],[15,4,2],[15,7,4]
], columns=['nameID', 'classID','rating'])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [47]:
import random

def get_top_n(predictions, n=10):
    # First map the predictions to each user.
    byUser = defaultdict(list)
    for p in predictions:
        byUser[p.uid].append(p)

    # For each user, reduce predictions to top-n
    for uid, userPredictions in byUser.items():
        byUser[uid] = heapq.nlargest(n, userPredictions, key=lambda p: p.est)
    return byUser

# Convert thes data set into the format required by the surprise package
# The columns must correspond to user id, item id and ratings (in that order)
reader = Reader(rating_scale=(1, 4))
data = Dataset.load_from_df(ratings[['nameID', 'classID', 'rating']], reader)

# Split into training and test set
trainset, testset = train_test_split(data, test_size=.25, random_state=1)

## User-based filtering
# compute cosine similarity between users
sim_options = {'name': 'cosine', 'user_based': True}
algo = KNNBasic(sim_options=sim_options)
algo.fit(trainset)


# Than predict ratings for all pairs (u, i) that are NOT in the training set.
predictions = algo.test(testset)

top_n = get_top_n(predictions, n=10)

# Print the recommended items for each user
print()
print('Top-3 recommended classes for each student')
for uid, user_ratings in list(top_n.items())[:15]:
    print('User {}'.format(uid))
    for prediction in user_ratings:
        print('  Item {0.iid} ({0.est:.2f})'.format(prediction), end='')
    print()
print()


## Item-based filtering
# compute cosine similarity between users
sim_options = {'name': 'cosine', 'user_based': False}
algo = KNNBasic(sim_options=sim_options)
algo.fit(trainset)

# Than predict ratings for all pairs (u, i) that are NOT in the training set.
predictions = algo.test(testset)
top_n = get_top_n(predictions, n=3)

# Print the recommended items for each user
print()
print('Top-3 recommended classes for each student')
for uid, user_ratings in list(top_n.items())[:15]:
    print('User {}'.format(uid))
    for prediction in user_ratings:
        print('  Item {0.iid} ({0.est:.2f})'.format(prediction), end='')
    print()

Computing the cosine similarity matrix...
Done computing similarity matrix.

Top-3 recommended classes for each student
User 5
  Item 2 (3.55)
User 15
  Item 7 (4.00)  Item 4 (3.55)
User 3
  Item 2 (3.55)  Item 1 (3.55)
User 4
  Item 7 (4.00)  Item 4 (3.55)
User 2
  Item 1 (4.00)

Computing the cosine similarity matrix...
Done computing similarity matrix.

Top-3 recommended classes for each student
User 5
  Item 2 (3.55)
User 15
  Item 7 (4.00)  Item 4 (3.55)
User 3
  Item 2 (3.55)  Item 1 (3.55)
User 4
  Item 4 (3.55)  Item 7 (3.50)
User 2
  Item 1 (4.00)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [79]:
## Build a model using the full dataset
trainset = data.build_full_trainset()
sim_options = {'name': 'cosine', 'user_based': False}
algo = KNNBasic(sim_options=sim_options)
algo.fit(trainset)

# Predict rating for student EN (4) and items (2, 3, 5, 6, 8)
algo.predict(4, 2)

Computing the cosine similarity matrix...
Done computing similarity matrix.


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Prediction(uid=4, iid=2, r_ui=None, est=4, details={'actual_k': 1, 'was_impossible': False})

In [40]:
from sklearn.metrics.pairwise import cosine_similarity

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [71]:
a = [[4]]
b = [[2]]
cosine_similarity(a, b)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([[1.]])